In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

# Competition을 위한 데이터 처리

In [2]:
mem_data = pd.read_csv('data/mem_data.csv')
mem_tr = pd.read_csv('data/mem_transaction.csv')

## 1. mem_data 처리

#### 1.1 SMS 수신동의 정수 처리하기

In [3]:
mem_data.SMS.value_counts()

Y    10575
N      189
Name: SMS, dtype: int64

In [4]:
mem_data.SMS = (mem_data.SMS=='Y').astype(int)
mem_data.SMS.value_counts()

1    10575
0      189
Name: SMS, dtype: int64

#### 1.2 양/음력(BIRTH_SL) 정수 처리하기 

In [5]:
mem_data.BIRTH_SL.value_counts()

S    8973
L    1791
Name: BIRTH_SL, dtype: int64

In [6]:
mem_data.BIRTH_SL = (mem_data.BIRTH_SL=='S').astype(int)
mem_data.BIRTH_SL.value_counts()

1    8973
0    1791
Name: BIRTH_SL, dtype: int64

#### 1.3 BIRTH_DT의 타입변환 및 년도 추출

In [7]:
mem_data['BIRTH_DT'] = mem_data['BIRTH_DT'].astype(str)
mem_data['BIRTH_DT'] = mem_data['BIRTH_DT'].apply(lambda x : x[:4] if len(x) == 10 else 'NaN')
mem_data['BIRTH_DT'] = mem_data['BIRTH_DT'].astype(float)
# YYYY-MM-DD의 형식을 만족하지 않는 생일 데이터는 4779개이다.
# 따라서 이외의 데이터는 모두 결측치로 판단한다.

In [8]:
#2007년 시점에서 2004년 이상의 고객은 잘못된 데이터로 간주 이외의 데이터는 0을 넣음
mem_data['BIRTH_DT'] = mem_data['BIRTH_DT'].apply(lambda x: x if x < 2004 else 0)
mem_data['BIRTH_DT'] = mem_data['BIRTH_DT'].astype(int)

#### 1.4 RGST_DT 등록일

In [9]:
mem_data['RGST_DT'] = pd.to_datetime(mem_data['RGST_DT'], format = '%Y-%m-%d %H:%M:%S')

#### 1.5 LAST_VST_DT(최근방문일자) 타입 변환

In [10]:
mem_data['LAST_VST_DT'] = pd.to_datetime(mem_data['LAST_VST_DT'], format = '%Y-%m-%d %H:%M:%S')

## 2. mem_data의 파생변수 생성

#### 2.1 최근 방문 일자(LAST_VST_DT)로부터 경과일 구하기 - 'DOR_DAY'(dormacy:휴먼)

In [11]:
visit = (pd.to_datetime('2007-12-31') - mem_data['LAST_VST_DT']).dt.days
mem_data['DOR_DAY'] = visit
mem_data['DOR_DAY'].head()

0    345
1     73
2    214
3     75
4    280
Name: DOR_DAY, dtype: int64

#### 2.2 등록일(RGST_DT)로부터 경과일 구하기 - 'VIS_DAY'

In [12]:
register = (pd.to_datetime('2007-12-31') - mem_data['RGST_DT']).dt.days
mem_data['VIS_DAY'] = register
mem_data['VIS_DAY'].head()

0    694
1    694
2    694
3    694
4    694
Name: VIS_DAY, dtype: int64

#### 2.3 2007년 기준으로 고객의 나이 변수 생성 -AGE

In [13]:
#### 한국 나이엔 +1 을 해야함.
mem_data['AGE'] = 2007 - mem_data['BIRTH_DT'] + 1

# 생일의 결측치인 0에 대한 값은 0으로 처리
mem_data['AGE'] = mem_data['AGE'].apply(lambda x: 0 if x == 2008 else x)
mem_data['AGE'].value_counts()[:5]

0     4803
25     405
26     395
28     373
24     371
Name: AGE, dtype: int64

#### 연령대가 0인 고객 = 결측치 이를  STORE의 연령대의 사람으로 대체

In [14]:
#각 STORE에 따른 고객 나이의 중간값
store_age = mem_data.groupby('M_STORE_ID')['AGE'].apply(lambda x: x.median()).reset_index()
store_age.rename(columns = {'AGE': 'PRED_AGE'}, inplace = True)
store_age = store_age.query('PRED_AGE ! = 0')
store_age.head()

,M_STORE_ID,PRED_AGE
0,125,21.5
1,126,22.0
2,128,2.5
3,130,22.0
13,456,29.0


In [15]:
mem_data = mem_data.merge(store_age, how = 'left', on = 'M_STORE_ID')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
mem_data['PRED_AGE'] = mem_data['PRED_AGE'].astype(int)

In [17]:
def replace_age_pred(x):
    pred_age_list = mem_data['PRED_AGE'].tolist()
    for i in pred_age_list:
        if x  < 1:
            return i
        else:
            return x

In [18]:
mem_data['F_AGE'] = mem_data['AGE'].apply(lambda x: replace_age_pred(x))

#### 2.4 연령대 더미변수- AGE_0, AGE_10, AGE_0, AGE_20, AGE_30, AGE_40,  AGE_50, AGE_A60

In [19]:
# 더미변수 생성을 위한 'AGE_G'칼럼 생성
def age_classifier(x):
    if x > 0 and x < 20:
        return '10'
    elif x >= 20 and x < 30:
        return '20'
    elif x >= 30 and x < 40:
        return '30'
    elif x >= 40 and x < 50:
        return '40'
    elif x >= 50 and x <60:
        return '50'
    elif x >= 60:
        return 'A_60' # 60이상의 데이터는 적기 때문에 60세 이상의 고객은 하나의 그룹으로 설정.
    else:
        return '0'

In [20]:
mem_data['AGE_G'] = mem_data['AGE'].apply(lambda x: age_classifier(x))

In [21]:
#각 연령대의 더미변수 생성
mem_data['AGE_0']= mem_data['AGE_G'].apply(lambda x: 1 if x == '0' else 0)
mem_data['AGE_10']= mem_data['AGE_G'].apply(lambda x: 1 if x == '10' else 0)
mem_data['AGE_20']= mem_data['AGE_G'].apply(lambda x: 1 if x == '20' else 0)
mem_data['AGE_30']= mem_data['AGE_G'].apply(lambda x: 1 if x == '30' else 0)
mem_data['AGE_40']= mem_data['AGE_G'].apply(lambda x: 1 if x == '40' else 0)
mem_data['AGE_50']= mem_data['AGE_G'].apply(lambda x: 1 if x == '50' else 0)
mem_data['AGE_A60']= mem_data['AGE_G'].apply(lambda x: 1 if x == '60' else 0)


#### 2.4 우편번호(ZIP_CD)에서 광역행정구역 칼럼 추가 - REGION

In [22]:
# 우편번호(ZIP_CD)에서 대부분의 행정구역은 '4'로 인천 또는 경기도에 거주하는 사람이다.
mem_data.query('ZIP_CD != "-"')['ZIP_CD'].str[:1].value_counts()

4    870
1    295
6    134
7     30
3     20
5     19
2      7
Name: ZIP_CD, dtype: int64

In [23]:
zip_cd = [x[0] for x in mem_data.ZIP_CD]
mem_data['REGION'] = zip_cd
mem_data['REGION'] = mem_data['REGION'].where(mem_data['REGION'] != '-', other=0).astype(int)
mem_data['REGION'].head()

0    0
1    4
2    0
3    4
4    0
Name: REGION, dtype: int32

#### 2.5 쇼핑몰에는 남자와 여자로 구분할 수 있을 것 - F_ST, M_ST

In [25]:
# 여성들이 많이 사용하는 쇼핑몰
f_store = mem_data.query('GENDER == "F"').groupby('M_STORE_ID')['MEM_ID'].agg([
    ('CNT', np.size)
]).reset_index()
f_store = f_store.sort_values(by = 'CNT', ascending = False)
f_store.head()

,M_STORE_ID,CNT
65,997,1431
168,2273,626
74,1113,146
167,2266,143
42,781,142


In [26]:
#남성들이 많이 사용하는 쇼핑몰
m_store = mem_data.query('GENDER == "M"').groupby('M_STORE_ID')['MEM_ID'].agg([
    ('CNT', np.size)
]).reset_index()
m_store = m_store.sort_values(by = 'CNT', ascending = False)
m_store.head()

,M_STORE_ID,CNT
87,997,212
132,1254,169
229,1870,122
92,1027,95
2,130,84


In [27]:
# 각 성별에 따른 상위 50개의 store_id를 집합 연산을 통해 여성중심의 스토어와 남성중심의 스토어를 찾아냄
a = set(f_store['M_STORE_ID'][:70].tolist())
b = set(m_store['M_STORE_ID'][:70].tolist())

# 차지합을 통해 여성, 남성 중심의 store_id list를 생성
#female = list(a-b)
#male = list(b-a)

In [28]:
mem_data['F_ST'] = mem_data['M_STORE_ID'].apply(lambda x: 1 if x in a else 0)
mem_data['M_ST'] = mem_data['M_STORE_ID'].apply(lambda x: 1 if x in b else 0)

#### 2.6 방문회수에 따른 구매금액 - SAL/VIST

In [29]:
# 구매 금액을 방문횟수로 나누어 방문회수당 구매금액 칼럼 추가
mem_data['SAL/VIST'] = round(mem_data['SALES_AMT'] / mem_data['VISIT_CNT'])

#### mem_data 변수 선택

In [30]:
#불필요한 변수 삭제
d_cols = ['M_STORE_ID', 'BIRTH_DT', 'BIRTH_SL', 'ZIP_CD', 'RGST_DT', 'LAST_VST_DT', 'AGE_G']

In [31]:
mem_data.drop(d_cols, axis = 1, inplace = True)
mem_data.head()

,MEM_ID,GENDER,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,DOR_DAY,...,AGE_10,AGE_20,AGE_30,AGE_40,AGE_50,AGE_A60,REGION,F_ST,M_ST,SAL/VIST
0,1134945,M,8.0,337000.0,6740.0,0.0,6740.0,3202.0,1,345,...,0,0,0,0,0,0,0,0,1,42125.0
1,38458,F,22.0,1438500.0,1732.0,12000.0,13732.0,17762.0,1,73,...,0,0,1,0,0,0,4,1,1,65386.0
2,7009,F,8.0,309300.0,6372.0,0.0,6372.0,1856.0,1,214,...,0,0,1,0,0,0,0,1,1,38662.0
3,91791,UNKNOWN,11.0,170400.0,3408.0,0.0,3408.0,236.0,1,75,...,0,0,0,1,0,0,4,1,1,15491.0
4,1374842,M,3.0,110300.0,2206.0,0.0,2206.0,1522.0,1,280,...,0,0,0,0,0,0,0,0,1,36767.0


##  4. mem_ transaction 데이터를 병합하여 변수 생성

#### 4.1 평균 구매액- BUY_MEAN

In [32]:
f = mem_tr.groupby('MEM_ID')['SELL_AMT'].agg([
    ('BUY_MEAN', lambda x:round(x.mean()) )
]).reset_index()

mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

#### 4.2 포인트 적립 횟수-POINT_N

In [33]:
f = mem_tr[mem_tr.MEMP_TP=='A'].groupby('MEM_ID')['SELL_AMT'].agg('size').reset_index()
f.columns = ['MEM_ID','POINT_N']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

#### 4.3 최빈 구매요일- MOST_FREQ_D

In [34]:
# SELL_DT를 datatime 타입으로 변환
mem_tr['SELL_DT'] = pd.to_datetime(mem_tr['SELL_DT'], format = '%Y%m%d')
# 요일을 의미하는 변수를 transaction으로 설정
mem_tr['D_WEEK'] = mem_tr['SELL_DT'].dt.weekday

#최빈 구매요일
f = mem_tr.groupby('MEM_ID')['D_WEEK'].agg([
    ('MOST_FREQ_D', lambda x: x.mode()[0])
]).reset_index()

mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

#### 4.4 주말, 주중 구매 횟수- WEEKEND_CNT, WEEK_CNT

In [35]:
mem_tr['WEEKEND_CNT'] = mem_tr['D_WEEK'].apply(lambda x: 1 if x >= 5 else 0)
mem_tr['WEEK_CNT'] = mem_tr['D_WEEK'].apply(lambda x: 1 if x < 5 else 0)

In [36]:
#주말 구매 횟수
f = mem_tr.groupby('MEM_ID')['WEEKEND_CNT'].sum().reset_index()
mem_data = mem_data.merge(f, how='left')

In [37]:
#주중 구매횟수
f = mem_tr.groupby('MEM_ID')['WEEK_CNT'].sum().reset_index()
mem_data = mem_data.merge(f, how='left')

#### 4.5 주말 중심 구매 고객인지  주중 중심 구매 인지 - WEEKEND_FREQ

In [38]:
def select_freq_w(x):
    w_cnt = mem_data['WEEK_CNT'].tolist()
    for i in w_cnt:
        if x > i:
            return 1
        else:
            return 0    

In [39]:
mem_data['WEEKEND_FREQ']= mem_data['WEEKEND_CNT'].apply(lambda x: select_freq_w(x))
mem_data.head()

,MEM_ID,GENDER,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,DOR_DAY,...,AGE_A60,REGION,F_ST,M_ST,SAL/VIST,BUY_MEAN,POINT_N,WEEKEND_CNT,WEEK_CNT,WEEKEND_FREQ
0,1134945,M,8.0,337000.0,6740.0,0.0,6740.0,3202.0,1,345,...,0,0,0,1,42125.0,16262,8,0,8,0
1,38458,F,22.0,1438500.0,1732.0,12000.0,13732.0,17762.0,1,73,...,0,4,1,1,65386.0,25018,22,3,19,0
2,7009,F,8.0,309300.0,6372.0,0.0,6372.0,1856.0,1,214,...,0,0,1,1,38662.0,22025,8,2,6,0
3,91791,UNKNOWN,11.0,170400.0,3408.0,0.0,3408.0,236.0,1,75,...,0,4,1,1,15491.0,14418,11,2,9,0
4,1374842,M,3.0,110300.0,2206.0,0.0,2206.0,1522.0,1,280,...,0,0,0,1,36767.0,11400,3,2,1,0


#### 4.6 시간대별 표인트 적립 건수 구하기 : Morning(09-12) / Afternoon(13-17) / Evening(18-20) - MORN, AFTR, EVEN

In [40]:
# MEMP_DT 변수를 datetime으로 변환
mem_tr['MEMP_DT'] = pd.to_datetime(mem_tr['MEMP_DT'], format = '%Y-%m-%d %H:%M:%S')

In [41]:
mem_tr['MEMP_HR'] = mem_tr['MEMP_DT'].dt.hour.apply(lambda x: 'M' if x >= 9 and x <= 12
                               else 'A' if x >= 13 and x <= 17
                               else 'E')

In [42]:
f = mem_tr.groupby('MEM_ID')['MEMP_HR'].agg([
    ('MEMP_HR', lambda x: x.mode()[0])
]).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

In [43]:
# 최빈 구매 시간 변수를 더미변수로 생성
mem_data['MORN']= mem_data['MEMP_HR'].apply(lambda x: 1 if x == 'M' else 0)
mem_data['AFTR']= mem_data['MEMP_HR'].apply(lambda x: 1 if x == 'A' else 0)
mem_data['EVEN']= mem_data['MEMP_HR'].apply(lambda x: 1 if x == 'E' else 0)

#### 4.7 계절에 따른 구매- TR_SP, TR_SM, TR_FL, TR_WT

In [44]:
# 각 거래 내역의 '월 칼럼 생성'
mem_tr['SELL_M'] = mem_tr['SELL_DT'].apply(lambda x: x.month)

In [45]:
# 고객별 계절에 따른 구매 건수
spring = mem_tr.query('SELL_M == 3 or SELL_M ==4 or SELL_M ==5').groupby('MEM_ID')['SELL_M'].count().reset_index()
spring.rename(columns = {'SELL_M' : 'TR_SP'}, inplace = True)

summer = mem_tr.query('SELL_M == 6 or SELL_M ==7 or SELL_M ==8').groupby('MEM_ID')['SELL_M'].count().reset_index()
summer.rename(columns = {'SELL_M' : 'TR_SM'}, inplace = True)

fall = mem_tr.query('SELL_M == 9 or SELL_M ==10 or SELL_M ==11').groupby('MEM_ID')['SELL_M'].count().reset_index()
fall.rename(columns = {'SELL_M' : 'TR_FL'}, inplace = True)

winter = mem_tr.query('SELL_M == 12 or SELL_M == 1 or SELL_M == 2').groupby('MEM_ID')['SELL_M'].count().reset_index()
winter.rename(columns = {'SELL_M' : 'TR_WT'}, inplace = True)

In [46]:
seasons = [spring, summer, fall, winter]
for i in seasons:
    mem_data = mem_data.merge(i, how = 'left', on ='MEM_ID')
    mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
    
mem_data[mem_data.columns[-4:].tolist()] = mem_data[mem_data.columns[-4:].tolist()].astype(int)

#### 4.8 의류시즌에 따른 구매 TR_S/S, TR_F/W

In [47]:
mem_data['TR_S/S'] = mem_data['TR_SP'] + mem_data['TR_SM']
mem_data['TR_F/W'] = mem_data['TR_FL'] + mem_data['TR_WT']

In [48]:
mem_data.head()

,MEM_ID,GENDER,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,DOR_DAY,...,MEMP_HR,MORN,AFTR,EVEN,TR_SP,TR_SM,TR_FL,TR_WT,TR_S/S,TR_F/W
0,1134945,M,8.0,337000.0,6740.0,0.0,6740.0,3202.0,1,345,...,E,0,0,1,2,1,4,1,3,5
1,38458,F,22.0,1438500.0,1732.0,12000.0,13732.0,17762.0,1,73,...,E,0,0,1,4,4,9,5,8,14
2,7009,F,8.0,309300.0,6372.0,0.0,6372.0,1856.0,1,214,...,E,0,0,1,5,1,1,1,6,2
3,91791,UNKNOWN,11.0,170400.0,3408.0,0.0,3408.0,236.0,1,75,...,A,0,1,0,1,3,5,2,4,7
4,1374842,M,3.0,110300.0,2206.0,0.0,2206.0,1522.0,1,280,...,M,1,0,0,2,0,0,1,2,1


#### 4.9 거래 정보- STY_O, STY_M

In [49]:
# 주문시 적립 횟수(O)
sty_o = mem_tr.query('MEMP_STY == "O"').groupby('MEM_ID')['MEMP_STY'].count().reset_index()
sty_o.rename(columns = {'MEMP_STY' : 'STY_O'}, inplace = True)

#추후 적립한 횟수(M)
sty_m = mem_tr.query('MEMP_STY == "M"').groupby('MEM_ID')['MEMP_STY'].count().reset_index()
sty_m.rename(columns = {'MEMP_STY' : 'STY_M'}, inplace = True)

In [50]:
styles = [sty_o, sty_m]
for i in styles:
    mem_data = mem_data.merge(i, how = 'left', on ='MEM_ID')
    mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
mem_data[mem_data.columns[-2:].tolist()] = mem_data[mem_data.columns[-2:].tolist()].astype(int)

### 불필요한 변수 삭제

In [51]:
mem_data.drop('MEMP_HR', axis = 1, inplace = True)

## 이상치 처리

In [52]:
mem_data['VISIT_CNT'] = mem_data['VISIT_CNT'].clip(upper = 300)
mem_data['SALES_AMT'] = mem_data['SALES_AMT'].clip(upper = 10000000)
mem_data['USABLE_PNT'] = mem_data['USABLE_PNT'].clip(upper = 70000)
mem_data['USED_PNT'] = mem_data['USED_PNT'].clip(upper = 150000)
mem_data['ACC_PNT'] = mem_data['ACC_PNT'].clip(upper = 140000)
mem_data['USABLE_INIT'] = mem_data['USABLE_INIT'].clip(upper = 30000)
mem_data['DOR_DAY'] = mem_data['DOR_DAY'].clip(upper = 680)
mem_data['VIS_DAY'] = mem_data['VIS_DAY'].clip(lower = 300)
mem_data['SAL/VIST'] = mem_data['SAL/VIST'].clip(lower = 100, upper = 1000000)
mem_data['BUY_MEAN'] = mem_data['BUY_MEAN'].clip(upper = 1000000)
mem_data['POINT_N'] = mem_data['POINT_N'].clip(upper = 400)
mem_data['WEEKEND_CNT'] = mem_data['WEEKEND_CNT'].clip(upper = 60)
mem_data['WEEK_CNT'] = mem_data['WEEK_CNT'].clip(upper = 300)

## 5. 이산형 변수들의 로그처리

#### 구매 합계(SALES_AMT) 로그 처리 하기
- 참고 : 로그처리에는 음수가 들어갈 수 없음 / 0이 있을 경우 1을 더하고 처리

In [53]:
def log_processing(x):
    f = mem_data[x].where(mem_data[x]>=0, other=0)
    f = np.log(f+1)
    mem_data[x] = f

In [54]:
log_target = ['SALES_AMT', 'USABLE_PNT', 'USED_PNT', 'ACC_PNT', 'USABLE_INIT',
              'DOR_DAY', 'VIS_DAY','SAL/VIST','BUY_MEAN']

In [55]:
for i in log_target:
    log_processing(i)

## **[최종 결과 저장]**

- mem_data에 최종 결과가 모일 수 있도록 준비
- 1.예측에 사용하지 않을 열들 drop
- 2.csv형태로 저장
- **주의사항1 : MEM_ID열은 삭제하지 마세요. **
- MEM_ID를 변수로 사용하기를 원하면 다른 열을 새로 생성해서 복사해서 사용하세요.
- 주의사항2 : GENDER열을 제외하고 문자열이 포함된 열이 데이터에 포함되지 않도록 한다.

In [56]:
mem_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10764 entries, 0 to 10763
Data columns (total 41 columns):
MEM_ID          10764 non-null int64
GENDER          10764 non-null object
VISIT_CNT       10764 non-null float64
SALES_AMT       10764 non-null float64
USABLE_PNT      10764 non-null float64
USED_PNT        10764 non-null float64
ACC_PNT         10764 non-null float64
USABLE_INIT     10764 non-null float64
SMS             10764 non-null int32
DOR_DAY         10764 non-null float64
VIS_DAY         10764 non-null float64
AGE             10764 non-null int64
PRED_AGE        10764 non-null int32
F_AGE           10764 non-null int64
AGE_0           10764 non-null int64
AGE_10          10764 non-null int64
AGE_20          10764 non-null int64
AGE_30          10764 non-null int64
AGE_40          10764 non-null int64
AGE_50          10764 non-null int64
AGE_A60         10764 non-null int64
REGION          10764 non-null int32
F_ST            10764 non-null int64
M_ST            10764 n

In [57]:
final_data = mem_data

In [58]:
final_data.to_csv('data_preparation_12.csv', index=False)